## Custom Sampling / Initial Population Creation

Creating a 2d array initial population.
1. no of weeks:4
2. no of category: 1
3. no of sku:5

so it will have the shape of 20, 6


**Skipped Constraints:**
1. Max promotions duration: 4 ie. the price options can only be 1 for at most 4 times consecutively # to remove? hard to model
2. Promotion Cooldown:2 there has to be a cooldown of 2 weeks # todo to remove??

**Modelled Constraints:**
1. Pricing options can only be all 0 or at most one 1.
2. promo upper limit per category: 3
3. display and feature can only be 1 if there is 1 pricing options true
4. display upper limit per category : 1
5. feature upper limit per category: 2


In [52]:
from pymoo.core.problem import ElementwiseProblem
from pymoo.optimize import minimize
from pymoo.algorithms.soo.nonconvex.ga import GA
import numpy as np
from pymoo.core.sampling import Sampling

from pymoo.operators.crossover.pntx import PointCrossover, SinglePointCrossover, TwoPointCrossover
from pymoo.operators.mutation.pm import PolynomialMutation
from pymoo.operators.repair.rounding import RoundingRepair

POP_SIZE = 10
CFG = {
    'sku_num': 5,                       # no of sku
    'h': 4,                             # week horizon
    'price_opt_num': 4,                 # num of pricing options dvar
    'ndf': 2,                           # num of display, feature dvar
    'lim_pro_per_cate_xu': 3,         # num of promotion items upper bound
    'lim_dis_per_cate_xu': 1,       # num of display items upper bound
    'lim_fea_per_cate_xu': 2,       # num of feature items upper bound
    'constraint_num':CFG["sku_num"] * CFG["h"]
}    

# (20, )
# PRICE_LIST = np.array([200, 190, 180, 170, 160, 150, 140, 130, 120, 110, 100, 90, 80, 70, 60, 50, 40, 30, 20, 10])
def init_price_list(x, y):
    size = x * y
    values = np.arange(size * 10, 0, -10)
    return values

PRICE_LIST = init_price_list(CFG["sku_num"], CFG["h"])

class SKUPopulationSampling(Sampling):
    """create intial population of all feasible solutions"""

    def __init__(self, cfg, pop_size):
        super().__init__()
        self.cfg = cfg
        self.pop_size = pop_size

    def _do(self, problem=None, n_samples=None, **kwargs):
        pop_size = self.pop_size
        nc = self.cfg['sku_num']
        h = self.cfg['h']
        price_ga = self.cfg['price_opt_num']
        ndf = self.cfg['ndf']
        lim_pro_per_cate_xu = self.cfg['lim_pro_per_cate_xu']
        lim_dis_per_cate_xu = self.cfg['lim_dis_per_cate_xu']
        lim_fea_per_cate_xu = self.cfg['lim_fea_per_cate_xu']
        
        # (population_size, no. of cate * weeks, no. of dvar per sku) 2, 10, 6
        # sku_pop = np.zeros((pop_size, nc*h, (price_ga + ndf)), dtype=int)
        # need to flatten for GA to work
        sku_pop = np.zeros((pop_size, nc*h * (price_ga + ndf)), dtype=int)
        

        for s in range(pop_size):
            x_sku_pop = np.zeros((nc*h, (price_ga + ndf)), dtype=int)
            
            d_nt = np.zeros(nc, dtype=int)
            f_nt = np.zeros(nc, dtype=int)
            # p_nt = np.zeros(nc, dtype=int)
            
            start_row = 0
            for pop_t in range(h):
                end_row = start_row + nc
             
                # no of promo per week at most Lim_promo_duration_upper
                sku_indices = np.random.permutation(nc)

                # Determine the number of SKUs on discount for this week
                num_discounted = min(lim_pro_per_cate_xu, nc)

                # Update pricing options for the selected SKUs
                for sku_idx in sku_indices[:num_discounted]:
                    discount_idx = np.random.choice(price_ga)
                    x_sku_pop[start_row + sku_idx, discount_idx] = 1
                
                # Update display and feature options only for SKUs on discount
                discounted_skus = sku_indices[:num_discounted]
                sample_d = np.random.choice(discounted_skus, min(lim_dis_per_cate_xu, num_discounted), replace=False)
                sample_f = np.random.choice(discounted_skus, min(lim_fea_per_cate_xu, num_discounted), replace=False)
                
                # Update the corresponding display and feature variables
                for sku_idx in discounted_skus:
                    if sku_idx in sample_d:
                        x_sku_pop[start_row + sku_idx, price_ga] = 1  # Display
                        d_nt[sku_idx] += 1
                    if sku_idx in sample_f:
                        x_sku_pop[start_row + sku_idx, price_ga + 1] = 1  # Feature
                        f_nt[sku_idx] += 1
                
                start_row = end_row
            
            # print(x_sku_pop)
            # populate solution to population
            # sku_pop[s, :, :] = x_sku_pop
            # need to flatten for GA to work
            sku_pop[s, :] = x_sku_pop.flatten()
        
        return sku_pop
    

class PromotionOptimizationProblem(ElementwiseProblem):
    def __init__(self):
        self.cfg = CFG
        self.sku_ndvar = (self.cfg["price_opt_num"]+self.cfg["ndf"]) # no of dvar per sku, cols
        self.rows = self.cfg["sku_num"] * self.cfg["h"] # no of rows in the 2d array of candidate soln
        n_var = self.rows * self.sku_ndvar

        super().__init__(n_var=n_var, n_obj=1, n_constr=self.cfg["constraint_num"], xl=np.zeros(n_var), xu=np.ones(n_var))



    def _evaluate(self, x, out, *args, **kwargs):


        can_sol = x.reshape((self.rows, self.sku_ndvar)) # candidate solution
        # print(f"{can_sol=}")

        ### CONSTRAINTS VIOLATION
        cv = np.sum(can_sol[:, :4], axis=1) - 1
        out["G"] = cv    

        # If there are any violations, set the objective value to a large negative value
        if np.any(cv > 0):
            out["F"] = -np.inf
            return
        
        ### PROFIT CALCULATION
        profit = 0
        # print(f"{x.shape=}, {x=}")
        discounted_values = np.zeros_like(PRICE_LIST)
        # print(f"xxxx {len(can_sol)}")
        # For loop to cater to demand func calculate
        for i in range(len(can_sol)):
            # If the first 4 values are all zeros, take the full price from price list
            if np.all(can_sol[i, :4] == 0):
                discounted_values[i] = PRICE_LIST[i]
            else:
                # Apply the corresponding discount based on the pattern in the price opt
                discount_factor = 0.8 if np.array_equal(can_sol[i, :4], [0, 0, 0, 1]) else \
                                0.6 if np.array_equal(can_sol[i, :4], [0, 0, 1, 0]) else \
                                0.4 if np.array_equal(can_sol[i, :4], [0, 1, 0, 0]) else \
                                0.2 if np.array_equal(can_sol[i, :4], [1, 0, 0, 0]) else 1
                discounted_values[i] = discount_factor * PRICE_LIST[i]

        # Calculate profit
        profit = np.sum(discounted_values)

        # print("Profit:", profit)
        constraints = []
        out["F"] = -profit
        # out["G"] = constraints

problem = PromotionOptimizationProblem()
algorithm = GA(pop_size=POP_SIZE, 
               sampling=SKUPopulationSampling(cfg=problem.cfg, pop_size=POP_SIZE),
               crossover=PointCrossover(prob=0.8, n_points=2),
               mutation = PolynomialMutation(prob=0.3, repair=RoundingRepair()),
               eliminate_duplicates=True
               )

res = minimize(problem,
               algorithm,
               ('n_gen', 10),
               seed=2,
               save_history=True,
               verbose=True)



n_gen  |  n_eval  |     cv_min    |     cv_avg    |     f_avg     |     f_min    
     1 |       10 |  0.000000E+00 |  0.000000E+00 | -1.474600E+03 | -1.638000E+03
     2 |       20 |  0.000000E+00 |  0.000000E+00 | -1.784000E+03 | -1.892000E+03
     3 |       30 |  0.000000E+00 |  0.000000E+00 | -1.896000E+03 | -1.988000E+03
     4 |       40 |  0.000000E+00 |  0.000000E+00 | -1.966600E+03 | -2.036000E+03
     5 |       50 |  0.000000E+00 |  0.000000E+00 | -2.027200E+03 | -2.056000E+03
     6 |       60 |  0.000000E+00 |  0.000000E+00 | -2.052400E+03 | -2.080000E+03
     7 |       70 |  0.000000E+00 |  0.000000E+00 | -2.071600E+03 | -2.080000E+03
     8 |       80 |  0.000000E+00 |  0.000000E+00 | -2.080000E+03 | -2.080000E+03
     9 |       90 |  0.000000E+00 |  0.000000E+00 | -2.080000E+03 | -2.080000E+03
    10 |      100 |  0.000000E+00 |  0.000000E+00 | -2.080000E+03 | -2.080000E+03


In [49]:
res.history

In [53]:
res.F

array([-2080.])

In [51]:
res.X

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1])

In [43]:
np.set_printoptions(threshold=np.inf)
res.pop.get("X")

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0,

## Draft Functions

In [24]:
import numpy as np

np.random.seed(42)

# Example arrays
# a = np.random.randint(0, 2, size=(20, 6))  # Example array a with shape (20, 6)
# b = np.array([200, 190, 180, 170, 160, 150, 140, 130, 120, 110, 100, 90, 80, 70, 60, 50, 40, 30, 20, 10])  # Example array b with shape (20,)

# a = np.random.randint(0, 2, size=(5, 6))  # Example array a with shape (20, 6)
a = np.array([
    [0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0],
    [0, 0, 1, 0, 0, 0],
    [0, 1, 0, 0, 0, 0],
    [1, 0, 0, 0, 0, 0],
])
b = np.array([200, 190, 180, 170, 160])  # Example array b with shape (20,)


print(a)
# Create a mask to identify rows where the first 4 values are all zero
zero_rows_mask = np.all(a[:, :4] == 0, axis=1)

# Create a mask for different discount scenarios
discount_masks = [
    np.all(a[:, :4] == np.array([0, 0, 0, 1]), axis=1),
    np.all(a[:, :4] == np.array([0, 0, 1, 0]), axis=1),
    np.all(a[:, :4] == np.array([0, 1, 0, 0]), axis=1),
    np.all(a[:, :4] == np.array([1, 0, 0, 0]), axis=1)
]

# Apply the corresponding discount to each row based on the mask
discounted_values = np.zeros_like(b)
discounted_values[zero_rows_mask] = b[zero_rows_mask]
discounted_values[discount_masks[0]] = 0.8 * b[discount_masks[0]]
discounted_values[discount_masks[1]] = 0.6 * b[discount_masks[1]]
discounted_values[discount_masks[2]] = 0.4 * b[discount_masks[2]]
discounted_values[discount_masks[3]] = 0.2 * b[discount_masks[3]]

# Calculate profit
profit = np.sum(discounted_values)

print("Profit:", profit)


[[0 0 0 0 0 0]
 [0 0 0 1 0 0]
 [0 0 1 0 0 0]
 [0 1 0 0 0 0]
 [1 0 0 0 0 0]]
Profit: 560
